In [1]:
import os

# Grabbing data with datalad
[Datalad](https://www.datalad.org/) is a git and git-annex based method of version control for data management, data sharing and reproducible science. For this lesson, it offers us a convenient way of grabbing the data that we need for the lesson. These comamands import datald so that we can use it to clone the repository into our virtual machine.

In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
import datalad.api as dl

In [4]:
dl_source='/Users/davecash/Data/IDEAS/sample'

In [5]:
sample=dl.clone(dl_source,path='/tmp/sample',description='Cloned sample dataset for import')

In [6]:
bl_img=sample.get('./baseline_t1.nii.gz')

In [7]:
fu_img=sample.get('./followup_t1.nii.gz')

# Running imaging with Nipype
The python package [nipype](https://nipype.readthedocs.io/en/latest/index.html) provides an effective means in Python to run your image processing workflows, taking "building blocks" of individual commands, allowing you to piece together bits of various software packages (FSL, SPM, FreeSurfer, AFNI, MRtrix, etc) into complete pipelines. We are just going to use it for simple cases here by registereing our baseline and followup image.

In [8]:
from nipype.interfaces import niftyreg


In [9]:
node = niftyreg.RegAladin()
node.inputs.ref_file = bl_img[0]['path']
node.inputs.flo_file = fu_img[0]['path']
out_path=os.path.dirname(bl_img[0]['path'])
node.inputs.aff_file = os.path.join(out_path,'followup_to_baseline_aff.txt')
node.inputs.res_file = os.path.join(out_path,'followup_resampled.nii.gz')
node.cmdline

'reg_aladin -aff /tmp/sample/followup_to_baseline_aff.txt -flo /tmp/sample/followup_t1.nii.gz -omp 1 -ref /tmp/sample/baseline_t1.nii.gz -res /tmp/sample/followup_resampled.nii.gz'

In [11]:
node.run()

210825-15:13:04,966 nipype.interface INFO:
	 stdout 2021-08-25T15:13:04.966098:[NiftyReg WARNING] NiftyReg has not been compiled with OpenMP, the '-omp' flag is ignored
210825-15:13:04,967 nipype.interface INFO:
	 stdout 2021-08-25T15:13:04.966098:[reg_aladin] 
210825-15:13:04,968 nipype.interface INFO:
	 stdout 2021-08-25T15:13:04.966098:[reg_aladin] Command line:
210825-15:13:04,968 nipype.interface INFO:
	 stdout 2021-08-25T15:13:04.966098:[reg_aladin] 	 reg_aladin -aff /tmp/sample/followup_to_baseline_aff.txt -flo /tmp/sample/followup_t1.nii.gz -omp 1 -ref /tmp/sample/baseline_t1.nii.gz -res /tmp/sample/followup_resampled.nii.gz
210825-15:13:04,969 nipype.interface INFO:
	 stdout 2021-08-25T15:13:04.966098:[reg_aladin] 
210825-15:13:04,970 nipype.interface INFO:
	 stdout 2021-08-25T15:13:04.966098:[reg_aladin_sym] Parameters
210825-15:13:04,971 nipype.interface INFO:
	 stdout 2021-08-25T15:13:04.966098:[reg_aladin_sym] Platform: cpu_platform
210825-15:13:04,971 nipype.interface INF

# FSL eyes setup

This *magic* command is needed to tie the wxWidgets backend used by FSLeyes to the notebook. Note that there are some steps to make this work locally, please consult the FSLeyes documentation about [linking to a Jupyter notebook](https://open.win.ox.ac.uk/pages/fsl/fsleyes/fsleyes/userdoc/fsleyes_notebook.html) and [troubleshooting for Mac](https://open.win.ox.ac.uk/pages/fsl/fsleyes/fsleyes/userdoc/troubleshooting.html#macos-i-can-t-start-fsleyes-from-ipython-jupyter-notebook) for more information.

In [10]:
%gui wx

Next, we initialise FSLeyes and create the frame, or window that we will be starting the viewer in.

In [11]:
import fsleyes
overlayList, displayCtx, frame = fsleyes.embed()
frame.Show()

15:00:29: Debug: Adding duplicate image handler for 'Windows bitmap file'
15:00:29: Debug: Adding duplicate animation handler for '1' type
15:00:29: Debug: Adding duplicate animation handler for '2' type
15:00:31: Debug: Adding duplicate image handler for 'Windows bitmap file'
15:00:31: Debug: Adding duplicate animation handler for '1' type
15:00:31: Debug: Adding duplicate animation handler for '2' type
/Users/davecash/anaconda3/envs/demon-imaging-harmonisation/lib/python3.7/site-packages/fsleyes/colourmaps.py:572: UserWarning: Trying to register the cmap 'pink' which already exists.
  mplcm.register_cmap(key, cmap, override_builtin=True)
/Users/davecash/anaconda3/envs/demon-imaging-harmonisation/lib/python3.7/site-packages/fsleyes/colourmaps.py:572: UserWarning: Trying to register the cmap 'hot' which already exists.
  mplcm.register_cmap(key, cmap, override_builtin=True)
/Users/davecash/anaconda3/envs/demon-imaging-harmonisation/lib/python3.7/site-packages/fsleyes/colourmaps.py:572:

True

Then we grab the image data that is needed for viewing. We first create an O

In [12]:
from fsleyes.views.orthopanel import OrthoPanel
from fsl.data.image import Image
ortho = frame.addViewPanel(OrthoPanel)

In [18]:
overlayList.append(Image(bl_img[0]['path']))
overlayList.append(Image(fu_img[0]['path']))

In [19]:
frame.Close()
fsleyes.shutdown()